# Tensor Networks - Matrix product States

In [1]:
import mps as mps
import numpy as np

We have seen that Quantum states are expressed as vectors of complex coefficients. The size of this vector grows exponentially with the number of particles $N$ as $2^N$ complex numbers. This means that storing a Quantum state of $N \sim 50$ may require PB of memory. 

Luckily, there is another way of writing Quantum states based on a matrix description. We assume that a Quantum state of N particles can be expressed in a basis of elements $|i_1 ... i_N\rangle$, where $i_k=[0,1]$ is an index. The basis has components for any combination of values on the indices. 

Let's take an arbitrary Quantum state
$$
\Psi = \sum c_{i_1,...i_N}|i_1 ... i-N\rangle.
$$

We have seen that we have $2^N$ coefficients $c_{i_1,...i_N}$, which makes this approach impractical. We may use the so-called Matrix Product State (MPS) form to write this set of coefficients $c_{i_1,...i_N}$ as a product of matrices:
$$
c_{i_1,...i_N} = Tr(M_{i_1} ... M_{i_N}),
$$
where we now have $2\times N$ matrices of size $D\times D$, and $Tr$ is the trace of a matrix. Thus, to recover i.e. $c_{010...1}$ we take the set of matrices $(M_0 ... M_{1})$, and after multiplying them we compute the trace. The resulting number is $c_{010...1}$.

The value of the matrices may change for each position. The parameter $D$ is an additional parameter that is adjusted to have an efficient representation: instead of using $2^N$ coefficients we have now $2\times D^2\times N$. The price we pay is that a representation of a Quantum state in MPS form may not be exact, only an approximation to the original state. 

The MPS form has many advantages and is used extensively in Quantum Theory. For the simulation of Quantum Computations, the main benefits are:

* The set of matrices $M_{i_1}$ can be stored is a distributed way
* The Quantum operation can be performed locally

For instance, we may want to apply a 1-qubit gate to the qubit in the position $k$. To perform this operation, we only need to use the matrices $M_0$ and $M_1$ at position $k$, and apply the 1-qubit gate. Afterwards, we recover the MPS form with local operations. For 2-qubit gates things can be slightly more complex, but alo may be perform locally.

We start definint an MPS of $5$ particles, and we pick the parameter $D=4$ as an example:

In [2]:
N_particles = 5
D = 4
mps_1 = mps.mps(N_particles, D)

We may recover the matrices of position $k$ from the MPS. We store these matrices in a data structure indexed by their position $k$, an the local index $i_k$ (thus we have $2\times N$ matrices of size $D\times D$):

In [3]:
k = 2
m0, m1 = mps_1.get_matrix(k)
print(m0)
print(m1)

[[0.38774978 0.5706682  0.80101153 0.90386621]
 [0.08812227 0.76674883 0.12545253 0.60408609]
 [0.62229941 0.2521343  0.4185775  0.47821025]
 [0.15391102 0.41633352 0.24851449 0.89753535]]
[[0.59061938 0.31226434 0.97947034 0.48586859]
 [0.14697797 0.75721574 0.39501722 0.64652322]
 [0.72911286 0.78057207 0.98960851 0.90033266]
 [0.56554279 0.05451761 0.23873149 0.5921678 ]]


We are assuming we already have an MPS form of our Quantum state. We don't cover there how to obtain these representation, although it is a straightforward operation.

Let's recover a couple of indices $c_{i_1,...i_N}$ of our Quantum state. To do so, we need to multiply a set of matrices and finally compute the trace. In this example, we take the coefficients $c_{01101}$ and $c_{00101}$.

In [4]:
print(mps_1.get_coeff([0,1,1,0,1]))
print(mps_1.get_coeff([0,0,1,0,1]))

31.75409518637929
34.21016123325102


We may now apply a 1-qubit gate at position $k$ to our Quantum state represented in MPS form. Let's use a simple 1-qubit gate that changes the value of the index $i_k$. The gate performing the swap operation can be expressed in matrix form as 
$$
S_x = \left( \begin{array}{cc}
0 & 1 \\
1 & 0
\end{array} \right)
$$

In [5]:
gate = np.array([[0,1],[1,0]])

To operate on our MPS, we combine this matrix $S_x$ only with those matrices $M_0$ and $M_1$ at position $k$:

In [6]:
mps_1.apply_gate(gate, 1)

1

After applying this operation, we have inverted the coefficient at position $k=1$, so the values of i.e. $c_{01101}$ and $c_{00101}$ should have swapped. We verify easily recovering again their values:

In [7]:
print(mps_1.get_coeff([0,1,1,0,1]))
print(mps_1.get_coeff([0,0,1,0,1]))

34.21016123325102
31.75409518637929
